# The Determinants of Turnout

In [20]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [21]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [22]:
%%stata

import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear

* Parse date
gen day = date(date, "YMD")
format day %td
gen month = month(day)
gen quarter = quarter(day)

gen year = year(day)
encode gecko_id, gen(token)
encode space,    gen(dao)

* Dependent variables
local dep non_whale_participation whale_participation

* Independent variables
local indep weighted quadratic ranked_choice quorum multi_choices have_discussion

* Label variables
label var weighted "\${\it Weighted}_{i,t}\$"
label var quadratic "\${\it Quadratic Voting}_{i,t}\$"
label var ranked_choice "\${\it Ranked Choice}_{i,t}\$"
label var quorum "\${\it Quorum}_{i,t}\$"
label var multi_choices "\${\it Multiple Choices}_{i,t}\$"
label var have_discussion "\${\it Discussion}_{i,t}\$"
label var delegation "\${\it Delegation}_{i,t}\$"


. 
. import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear
(encoding automatically selected: ISO-8859-1)
(32 vars, 2,841 obs)

. 
. * Parse date
. gen day = date(date, "YMD")

. format day %td

. gen month = month(day)

. gen quarter = quarter(day)

. 
. gen year = year(day)

. encode gecko_id, gen(token)

. encode space,    gen(dao)

. 
. * Dependent variables
. local dep non_whale_participation whale_participation

. 
. * Independent variables
. local indep weighted quadratic ranked_choice quorum multi_choices have_discus
> sion

. 
. * Label variables
. label var weighted "\${\it Weighted}_{i,t}\$"

. label var quadratic "\${\it Quadratic Voting}_{i,t}\$"

. label var ranked_choice "\${\it Ranked Choice}_{i,t}\$"

. label var quorum "\${\it Quorum}_{i,t}\$"

. label var multi_choices "\${\it Multiple Choices}_{i,t}\$"

. label var have_discussion "\${\it Discussion}_{i,t}\$"

. label var delegation "\${\it Delegation}_{i,t}\$"

. 


## Proposal Characteristics

In [23]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

foreach d of local dep {
    * Run regression
    reghdfe `d' `indep', absorb(year dao)
    eststo `d'

    * Tag FE rows (Y/N) for this spec
    estadd local fe_token "Y"
    estadd local fe_time  "Y"
}

* Export LaTeX table
esttab                                                      ///
    non_whale_participation whale_participation   ///
    using "$TABLES/reg_participation_char.tex", replace           ///
    se star(* 0.10 ** 0.05 *** 0.01) nogaps nocon           ///
    b(%9.3f) se(%9.2f)                                      ///
    label nogaps booktabs nonotes noomitted eqlabels(none)  ///
    mtitles("\${\it Participation}^{Small}_{i,t}\$"               ///
    "\${\it Participation}^{Whale}_{i,t}\$")                      ///  
    substitute("\_" "_")                                    ///
    stats(fe_token fe_time N r2_a,                          ///
        fmt(0 0 %9.0fc %9.3f)                                   ///
         labels("Token FE" "Year FE" "Observations" "Adjusted R²"))


. 
. ******************************************************
. * LOGISTIC REGRESSIONS
. ******************************************************
. 
. eststo clear

. 
. foreach d of local dep {
  2.     * Run regression
.     reghdfe `d' `indep', absorb(year dao)
  3.     eststo `d'
  4. 
.     * Tag FE rows (Y/N) for this spec
.     estadd local fe_token "Y"
  5.     estadd local fe_time  "Y"
  6. }
(dropped 12 singleton observations)
(MWFE estimator converged in 7 iterations)

HDFE Linear regression                            Number of obs   =      2,667
Absorbing 2 HDFE groups                           F(   6,   2582) =      14.04
                                                  Prob > F        =     0.0000
                                                  R-squared       =     0.5492
                                                  Adj R-squared   =     0.5345
                                                  Within R-sq.    =     0.0316
                                            